In [1]:
import sys 
sys.path.append('../')
from utils.llm import LLM
import tqdm

llm = LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [2]:
query = "パソコンのキーボードの上部についている、「F1」「F2」などと書かれているキーの総称は何でしょう?"
answer = llm.generate(query)
print(answer)

ファンクションキーです。 F1、F2などのキーを押すと、画面に表示されるさまざまな機能を呼び出すことができます。


In [3]:
import pprint
from load_JAQKET import load_jaqket

dataset = load_jaqket(n_samples=250)
pprint.pprint(dataset[:2])

[{'answer': 'クリオネ',
  'qid': 'QA20CAPR-0001',
  'question': '和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる動物は何でしょう?'},
 {'answer': '東方見聞録',
  'qid': 'QA20CAPR-0003',
  'question': '作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?'}]


In [4]:
import pandas as pd
import tqdm 

df = pd.DataFrame(dataset)
# colとしてgenerated_answerを追加
df["generated_answer"] = None

for d in tqdm.tqdm(dataset):
    question = d["question"]
    generated_answer = llm.generate(question, temperture=0.01)

    df.loc[df["qid"] == d["qid"], "generated_answer"] = generated_answer


100%|██████████| 250/250 [11:37<00:00,  2.79s/it]


In [5]:
df

,qid,question,answer,generated_answer
0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,クリオネ,マンタ
1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,東方見聞録,『東方見聞録』です。
2,QA20CAPR-0005,『騎士団長殺し』『1Q84』『ノルウェイの森』といった小説の作者は誰でしょう?,村上春樹,村上春樹
3,QA20CAPR-0007,サッカーで、コーナーキックやフリーキックのようにボールを止めた状態から始めるプレーのことを何...,セットプレー,ゴールキック(Goal Kick)といいます。
4,QA20CAPR-0009,今から約140億年前に起こったとされる、宇宙の始まりの大爆発のことを何というでしょう?,ビッグバン,ビッグバン
...,...,...,...,...
245,QA20CAPR-0495,その職は北条氏が独占した、鎌倉幕府において将軍を補佐する役職を何というでしょう?,執権,執権職(しっけんしょく)。
246,QA20CAPR-0497,リーフデ号に乗って日本に漂着し、徳川家康に外交顧問として仕えたイギリスの航海士は誰でしょう?,ウィリアム・アダムス,ジョン・アダムズ
247,QA20CAPR-0499,全メソポタミアの統一に成功したバビロン第一王朝の王で、「目には目を、歯には歯を」で有名な法典...,ハンムラビ,ハンムラビ法典
248,QA20CAPR-0501,フランス語で「千枚の葉」という意味がある、薄いパイとカスタードクリームを何枚も重ねたフランス...,ミルフィーユ,フォレ・ノワール


In [6]:
df = df.dropna(subset=["generated_answer"])

In [7]:
# dfを保存
df.to_csv("df_JAQKET_qa.csv", index=False)

In [1]:
import pandas as pd
import sys 
sys.path.append('../')
from utils.scoring import sentence_splitter

df = pd.read_csv("df_JAQKET_qa.csv")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class y

In [2]:
df["num_sentences"] = df["generated_answer"].apply(lambda x: len(sentence_splitter(x)))

In [3]:
df.head()

,qid,question,answer,generated_answer,num_sentences
0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,クリオネ,マンタ,1
1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,東方見聞録,『東方見聞録』です。,1
2,QA20CAPR-0005,『騎士団長殺し』『1Q84』『ノルウェイの森』といった小説の作者は誰でしょう?,村上春樹,村上春樹,1
3,QA20CAPR-0007,サッカーで、コーナーキックやフリーキックのようにボールを止めた状態から始めるプレーのことを何...,セットプレー,ゴールキック(Goal Kick)といいます。,1
4,QA20CAPR-0009,今から約140億年前に起こったとされる、宇宙の始まりの大爆発のことを何というでしょう?,ビッグバン,ビッグバン,1


In [4]:
df.describe()

,num_sentences
count,250.000000
mean,1.328000
std,0.833785
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,6.000000


In [10]:
df.to_csv("df_JAQKET_qa.csv", index=False)